# Generating a QRcode in a Terminal window
---

Sometimes nothing else will work but a QRcode (e.g. for sending a URL to a mobile device).

This little script will generate a QRcode in a terminal window.

In [ ]:
require 'rqrcode'
require 'term/ansicolor'
require 'highline/import'
require 'optparse'
require 'cgi'

text_to_encode = "hello world"

@size     = 4

class String
  include Term::ANSIColor

  # my terminal is black background, so space comes out as black and to have
  # white, need to have bright-white.
  def dark_if(yes_or_no)
    yes_or_no ? '  ' : on_white.white.bold
  end
end

qrcode = nil

# Need to the size of the qrcode, this is depended on the length
# of what we're encoding. This code determines the size automagically.
50.times do
  begin
    qrcode = RQRCode::QRCode.
      new(text_to_encode, :size => @size.to_i, :level => 'q')
  rescue RQRCode::QRCodeRunTimeError => e
    @size = (@size.to_i + 1).to_s
  end
end

ary_size =  qrcode.module_count + 2
ary = Array.new(ary_size) { |_| [false]*ary_size }

qrcode.modules.each_index do |x|
  qrcode.modules.each_index do |y|
    ary[x+1][y+1] = qrcode.dark?(x,y)
  end
end

# because of the font setting on my terminal, to get a square qrcode, need to use
# ██ instead of just █ ...
(0...ary.size).to_a.tap do |loopidx|
  loopidx.each do |xidx|
    loopidx.each do |yidx|
      print '██'.dark_if(ary[xidx][yidx])
    end
    puts ""
  end
end

nil